In [ ]:
%pylab inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

In [ ]:
print(data['feature_names'])
print(len(data['feature_names']))

In [ ]:
print(data['target_names'])
print(len(data['target_names']))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = data['data']
print(X[0])

# Standerdization（標準化）
#X = scaler.fit_transform(X)
#print(X[0])

In [ ]:
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(np.arange(len(X)), test_size=0.3, random_state=42)
train_X = torch.tensor(X[train_idx], dtype=torch.float)
train_y = torch.tensor(data['target'][train_idx], dtype=torch.long)
test_X = torch.tensor(X[test_idx], dtype=torch.float)
test_y = torch.tensor(data['target'][test_idx], dtype=torch.long)

In [ ]:
class MLP(nn.Module):
    def __init__(self, xdim, hdim, ydim):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(xdim, hdim)
        self.linear2 = nn.Linear(hdim, ydim)
        self.norm1 = nn.BatchNorm1d(hdim)
        self.dropout1 = nn.Dropout(p=0.2)
        
    def forward(self, X):
        a1 = self.linear1(X)
        
        # Batch normalization（バッチ正規化）
        #a1 = self.norm1(a1)
        
        # Dropout
        #a1 = self.dropout1(a1)
        
        z1 = F.relu(a1)
        a2 = self.linear2(z1)
        return a2

In [ ]:
mlp = MLP(30, 5, 2)
a2 = mlp(train_X)

In [ ]:
mlp = MLP(30, 5, 2)

optimizer = optim.SGD(mlp.parameters(), lr = 0.1)

# Weight decay（重み減衰あり）
# optimizer = optim.SGD(mlp.parameters(), lr = 0.1, weight_decay=0.01)

# AdaGrad
#optimizer = optim.Adagrad(mlp.parameters())

# RMSProp
#optimizer = optim.RMSprop(mlp.parameters())

# Adam
#optimizer = optim.Adam(mlp.parameters())

for epoch in range(1000):
    optimizer.zero_grad()
    a = mlp(train_X)
    loss = F.cross_entropy(a, train_y)
    loss.backward()
    optimizer.step()
    print(loss)

In [ ]:
mlp.eval()
a = mlp(test_X)
pred_proba = F.softmax(a, dim=1)
pred_y = torch.argmax(pred_proba, dim=1)
print(pred_y)
accuracy = (pred_y == test_y).sum() / float(len(pred_y))
print(accuracy)